# Weighted Moving Average

## Sample

In [1]:
import numpy as np
import os
import sys
import pandas as pd
d = pd.DataFrame({'a':range(10), 'b':np.random.random(size=10)})
d.b = d.b.round(2)
print(d)

   a     b
0  0  0.19
1  1  0.00
2  2  0.91
3  3  0.16
4  4  0.67
5  5  0.13
6  6  0.80
7  7  0.04
8  8  0.37
9  9  0.86


In [5]:
wts = np.array([-1, 2])
def f(w):                        
        def g(x):
            return (w*x).mean()
        return g
d["C"] = d["a"].rolling(window=2).apply(f(wts))
d["D"] = d["b"].rolling(window=2).apply(f(wts))
print(type(wts))
print(wts)
print(d)

<class 'numpy.ndarray'>
[-1  2]
   a     b    C      D
0  0  0.19  NaN    NaN
1  1  0.00  1.0 -0.095
2  2  0.91  1.5  0.910
3  3  0.16  2.0 -0.295
4  4  0.67  2.5  0.590
5  5  0.13  3.0 -0.205
6  6  0.80  3.5  0.735
7  7  0.04  4.0 -0.360
8  8  0.37  4.5  0.350
9  9  0.86  5.0  0.675


## Skyze Implementation

In [49]:
import numpy as np
import os
import sys
import pandas as pd
import settings_skyze

file_path = os.path.join(settings_skyze.test_data_file_path, "%s.csv" % "bitcoin_TEST")
df = pd.read_csv(
                        "../"+file_path, 
                        header=None ,
                        names = [
                                  "Date", "Open", "High", "Low",
                                  "Close", "Volume", "MarketCap","HLOrder"
                                ],
                        #index_col = "Date"
                        )
print(df.head(5))
import datetime as datetime
from dateutil import parser
df.index = [parser.parse(str(d)) for d in df["Date"]]
del df["Date"]
#df.index = pd.to_datetime(df.index)
p_data = df
print(p_data.head(5))

       Date    Open    High     Low   Close Volume   MarketCap  HLOrder
0  20130428  135.30  135.98  132.10  134.21      -  1500520000      NaN
1  20130429  134.44  147.49  134.00  144.54      -  1491160000      NaN
2  20130430  144.00  146.93  134.05  139.00      -  1597780000      NaN
3  20130501  139.00  139.89  107.72  116.99      -  1542820000      NaN
4  20130502  116.38  125.60   92.28  105.21      -  1292190000      NaN
              Open    High     Low   Close Volume   MarketCap  HLOrder
2013-04-28  135.30  135.98  132.10  134.21      -  1500520000      NaN
2013-04-29  134.44  147.49  134.00  144.54      -  1491160000      NaN
2013-04-30  144.00  146.93  134.05  139.00      -  1597780000      NaN
2013-05-01  139.00  139.89  107.72  116.99      -  1542820000      NaN
2013-05-02  116.38  125.60   92.28  105.21      -  1292190000      NaN


In [75]:
# Option 2 using Pandas Rolling
        # https://stackoverflow.com/questions/39742797/calculating-weighted-moving-average-using-pandas-rolling-method
        # construct thet weights array
wma_period = 15
weights = np.array(list(range(1,wma_period+1)))
print(type(weights))
print(weights)

<class 'numpy.ndarray'>
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [87]:
# construct the fucntion to calcualte wma for each point
def wma(w):
    def g(x):
        return (w/x).mean()
    return g

print(type(wma))
print(wma)
print(type(wma(weights)))
print(wma(weights))

<class 'function'>
<function wma at 0x10ce60c80>
<class 'function'>
<function wma.<locals>.g at 0x10ce608c8>


In [88]:
wma_column = "Close"
p_data["WMA_15"]=p_data[wma_column].rolling(window=wma_period).apply(wma((weights,)))

p_data["roll_app"]=pd.rolling_apply(p_data[wma_column],wma_period,wma(weights))


/Users/michaelnew/.local/share/virtualenvs/Skyze-v-C51IT9/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pd.rolling_apply is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=15,center=False).apply(func=<function>,args=<tuple>,kwargs=<dict>)
  after removing the cwd from sys.path.


In [89]:
print(p_data[["Close","WMA_15","roll_app"]].head(25))

             Close    WMA_15  roll_app
2013-04-28  134.21       NaN       NaN
2013-04-29  144.54       NaN       NaN
2013-04-30  139.00       NaN       NaN
2013-05-01  116.99       NaN       NaN
2013-05-02  105.21       NaN       NaN
2013-05-03   97.75       NaN       NaN
2013-05-04  112.50       NaN       NaN
2013-05-05  115.91       NaN       NaN
2013-05-06  112.30       NaN       NaN
2013-05-07  111.50       NaN       NaN
2013-05-08  113.57       NaN       NaN
2013-05-09  112.67       NaN       NaN
2013-05-10  117.20       NaN       NaN
2013-05-11  115.24       NaN       NaN
2013-05-12  115.00  0.070216  0.070216
2013-05-13  117.98  0.070104  0.070104
2013-05-14  111.50  0.070416  0.070416
2013-05-15  114.22  0.070378  0.070378
2013-05-16  118.76  0.069901  0.069901
2013-05-17  123.02  0.069141  0.069141
2013-05-18  123.50  0.068441  0.068441
2013-05-19  121.99  0.067984  0.067984
2013-05-20  122.00  0.067572  0.067572
2013-05-21  122.88  0.067130  0.067130
2013-05-22  123.89  0.066

In [54]:
print((134+115*15)/2)

929.5


In [57]:
x=p_data[wma_column].rolling(window=wma_period)
print(type(x))
print(x)

<class 'pandas.core.window.Rolling'>
Rolling [window=15,center=False,axis=0]


In [62]:
print(type(x.apply(wma(weights))))
print(x.apply(wma(weights)).head(20))

<class 'pandas.core.series.Series'>
2013-04-28         NaN
2013-04-29         NaN
2013-04-30         NaN
2013-05-01         NaN
2013-05-02         NaN
2013-05-03         NaN
2013-05-04         NaN
2013-05-05         NaN
2013-05-06         NaN
2013-05-07         NaN
2013-05-08         NaN
2013-05-09         NaN
2013-05-10         NaN
2013-05-11         NaN
2013-05-12    0.070216
2013-05-13    0.070104
2013-05-14    0.070416
2013-05-15    0.070378
2013-05-16    0.069901
2013-05-17    0.069141
Name: Close, dtype: float64


In [68]:
list(range(1,16))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [64]:
print(range(15))

range(0, 15)
